<a href="https://colab.research.google.com/github/bibekebib/Major-project-raw-files/blob/main/Keypoint_Extraction_testing_on_original_dataset.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
dataset_path = '/content/drive/MyDrive/dataset'

In [3]:
from glob import glob
import pandas as pd
import cv2 as cv2
import matplotlib.pyplot as plt
import os
import numpy as np
import torch
from sklearn.model_selection import train_test_split
!pip install mediapipe

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 33.5/33.5 MB 31.9 MB/s eta 0:00:00


In [4]:
main_df = pd.read_csv(dataset_path+'/df_original.csv')

In [5]:
main_df.Label.value_counts()

bhujan      16
vriksh      15
tadasana    15
shav        15
padmasan    14
trik        13
Name: Label, dtype: int64

In [6]:
X = main_df.Path
y = main_df.Label

In [7]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=1, stratify=y)

In [8]:
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.25, stratify=y_train)

In [9]:
# y_train

In [10]:
df = pd.concat((X_train, y_train), axis=1).reset_index().drop('index', axis=1)
testdf = pd.concat((X_test, y_test), axis=1).reset_index().drop('index', axis=1)
valdf = pd.concat((X_val, y_val), axis=1).reset_index().drop('index', axis=1)

In [11]:
df.Label.value_counts(), testdf.Label.value_counts(),valdf.Label.value_counts()

(bhujan      10
 shav         9
 vriksh       9
 tadasana     9
 padmasan     8
 trik         7
 Name: Label, dtype: int64,
 tadasana    3
 vriksh      3
 padmasan    3
 shav        3
 trik        3
 bhujan      3
 Name: Label, dtype: int64,
 trik        3
 shav        3
 tadasana    3
 padmasan    3
 bhujan      3
 vriksh      3
 Name: Label, dtype: int64)

In [12]:
import mediapipe as mp
mp_drawing = mp.solutions.drawing_utils
mp_drawing_styles = mp.solutions.drawing_styles
mp_pose = mp.solutions.pose

In [13]:
BG_COLOR = (192, 192, 192)
def landmark_det(image):
  with mp_pose.Pose(
    static_image_mode=True,
    model_complexity=2,
    enable_segmentation=True,
    min_detection_confidence=0.5) as pose:
    a = []
    image_height, image_width, _ = image.shape
    results = pose.process(cv2.cvtColor(image, cv2.COLOR_BGR2RGB))
    for i in range(33):
      a.append((results.pose_landmarks.landmark[mp_pose.PoseLandmark(i)].x ,results.pose_landmarks.landmark[mp_pose.PoseLandmark(i)].y ,results.pose_landmarks.landmark[mp_pose.PoseLandmark(i)].z))
    b = []

    for i in a:
      for x in i:
        b.append(x)
    df = pd.DataFrame(b).T
    df = df.to_numpy()
    return df

In [14]:
# data =  landmark_det(cv2.imread('/content/aa.webp'))

In [15]:
# data

In [16]:
feat = ['NOSE',
'LEFT_EYE_INNER',
'LEFT_EYE',
'LEFT_EYE_OUTER',
'RIGHT_EYE_INNER',
'RIGHT_EYE',
'RIGHT_EYE_OUTER',
'LEFT_EAR',
'RIGHT_EAR',
'MOUTH_LEFT',
'MOUTH_RIGHT',
'LEFT_SHOULDER',
'RIGHT_SHOULDER',
'LEFT_ELBOW',
'RIGHT_ELBOW',
'LEFT_WRIST',
'RIGHT_WRIST',
'LEFT_PINKY',
'RIGHT_PINKY',
'LEFT_INDEX',
'RIGHT_INDEX',
'LEFT_THUMB',
'RIGHT_THUMB',
'LEFT_HIP',
'RIGHT_HIP',
'LEFT_KNEE',
'RIGHT_KNEE',
'LEFT_ANKLE',
'RIGHT_ANKLE',
'LEFT_HEEL',
'RIGHT_HEEL']

In [17]:
import os
import time

parentFolderTrain = '/content/drive/MyDrive/OriginalFramesTrain'
parentFolderTest = '/content/drive/MyDrive/OriginalFolderTest'
def listframe(parentFolder,path, Label):
  local = []
  os.chdir(parentFolder)
  foldername = (Label).lower()
  # if os.path.exists(parentFolder+'/'+foldername):
  #   os.chdir(foldername)
  # else:
  #   # os.mkdir(foldername)
  #   os.chdir(foldername)
  path_fol = Label+'-'+path.split('/')[-1].split('.')[0]
  # print(os.getcwd())
  # if os.path.exists(parentFolder+'/'+path_fol):
  #   os.chdir(path_fol)
  # else:
  #   # os.rmdir(path_fol)
  #   os.mkdir(path_fol)
  #   os.chdir(path_fol)
  video = cv2.VideoCapture(path)
  frame_num = int(video.get(cv2.CAP_PROP_FRAME_COUNT))
  # print(frame_num)
  # print(int(frame_num /64))
  f_img = 0
  # print(os.getcwd())
  count = 0
  while (count<frame_num):
      try:
          success, cap = video.read()
          if count % (int(frame_num / 256)) == 0 and (f_img<256):
              # print(count)
              # plt.imshow(cap)
              # plt.show()

              # cv2.imwrite(f" {Label+'-'+path.split('/')[-1].split('.')[0]} frame%d.jpg" % f_img, cap) # save frame as JPEG file.
              # frame_height, frame_width = cap.shape[:2]
              df  = landmark_det(cap)
              local.append([df, Label])
              # print(local)
              f_img =f_img+1
              count = count+1
          else:
              count = count+1

      except:
        df = [np.nan] * 99
        local.append([df, Label])
        count = count+1
        print(path)
        print(count)
  print(f'done dong doing {path}')
  # print(local)
  return local


In [ ]:
# testdf= testdf.sample(frac=1).reset_index()

In [20]:
# ldata = (listframe(parentFolderTrain,'/content/drive/MyDrive/dataset/tadasana/Piyush_Tad.mp4', df.Label[10]))
# ldata

In [21]:

# (ldata)

In [22]:
import tqdm
globals = []

for i in range(len(df.Path)):
  globals.append(listframe(parentFolderTrain,df.Path[i], df.Label[i]))

Exception in thread _colab_inspector_thread:
Traceback (most recent call last):
  File "/usr/lib/python3.10/threading.py", line 1016, in _bootstrap_inner
    self.run()
  File "/usr/lib/python3.10/threading.py", line 953, in run
    self._target(*self._args, **self._kwargs)
  File "/usr/local/lib/python3.10/dist-packages/google/colab/_debugpy.py", line 64, in inspector_thread
    _variable_inspector.run(shell, time)
  File "/usr/local/lib/python3.10/dist-packages/google/colab/_variable_inspector.py", line 27, in run
    globals().clear()
TypeError: 'list' object is not callable


done dong doing /content/drive/MyDrive/dataset/bhujan/Harshav_Bhuj.mp4
/content/drive/MyDrive/dataset/bhujan/Veena_Bhu.mp4
326
/content/drive/MyDrive/dataset/bhujan/Veena_Bhu.mp4
331
/content/drive/MyDrive/dataset/bhujan/Veena_Bhu.mp4
336
done dong doing /content/drive/MyDrive/dataset/bhujan/Veena_Bhu.mp4
/content/drive/MyDrive/dataset/shav/deepa_shava.mp4
5
/content/drive/MyDrive/dataset/shav/deepa_shava.mp4
49
/content/drive/MyDrive/dataset/shav/deepa_shava.mp4
53
/content/drive/MyDrive/dataset/shav/deepa_shava.mp4
65
/content/drive/MyDrive/dataset/shav/deepa_shava.mp4
77
/content/drive/MyDrive/dataset/shav/deepa_shava.mp4
85
/content/drive/MyDrive/dataset/shav/deepa_shava.mp4
93
/content/drive/MyDrive/dataset/shav/deepa_shava.mp4
113
/content/drive/MyDrive/dataset/shav/deepa_shava.mp4
133
/content/drive/MyDrive/dataset/shav/deepa_shava.mp4
137
/content/drive/MyDrive/dataset/shav/deepa_shava.mp4
157
/content/drive/MyDrive/dataset/shav/deepa_shava.mp4
161
/content/drive/MyDrive/datase

In [23]:
final = np.array(globals)
np.save('final_256.npy', final)

<ipython-input-23-0fcd526c3b45>:1: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  final = np.array(globals)


In [24]:
np.save('/content/drive/MyDrive/final_256.npy', final)

In [25]:
import tqdm
globals = []

for i in range(len(valdf.Path)):
  globals.append(listframe(parentFolderTest,valdf.Path[i], valdf.Label[i]))

done dong doing /content/drive/MyDrive/dataset/trik/Shiva_Trik.mp4
done dong doing /content/drive/MyDrive/dataset/trik/deepa_trikon.mp4
done dong doing /content/drive/MyDrive/dataset/trik/Pranshul_Trik.mp4
/content/drive/MyDrive/dataset/shav/Shiva_Shavasana.mp4
251
/content/drive/MyDrive/dataset/shav/Shiva_Shavasana.mp4
311
/content/drive/MyDrive/dataset/shav/Shiva_Shavasana.mp4
1206
/content/drive/MyDrive/dataset/shav/Shiva_Shavasana.mp4
1211
done dong doing /content/drive/MyDrive/dataset/shav/Shiva_Shavasana.mp4
done dong doing /content/drive/MyDrive/dataset/tadasana/Santosh_Tada.mp4
/content/drive/MyDrive/dataset/tadasana/Piyush_Tad.mp4
65
/content/drive/MyDrive/dataset/tadasana/Piyush_Tad.mp4
69
/content/drive/MyDrive/dataset/tadasana/Piyush_Tad.mp4
73
/content/drive/MyDrive/dataset/tadasana/Piyush_Tad.mp4
89
/content/drive/MyDrive/dataset/tadasana/Piyush_Tad.mp4
93
/content/drive/MyDrive/dataset/tadasana/Piyush_Tad.mp4
101
/content/drive/MyDrive/dataset/tadasana/Piyush_Tad.mp4
105

In [26]:
final = np.array(globals)
np.save('final_val_256.npy', final)
np.save('/content/drive/MyDrive/final_val_256.npy', final)

<ipython-input-26-1edf360e29af>:1: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  final = np.array(globals)


In [ ]:
# import tqdm
# for i in range(len(testdf.Path)):
#   tqdm.tqdm(listframe(parentFolderTest,testdf.Path[i], testdf.Label[i]))

In [ ]:
# import os
# import time
# parentFolder = '/content/drive/MyDrive/SagarVideosEachFrameFolder'
# def listframe(path, Label):
#   os.chdir(parentFolder)
#   foldername = (Label).lower()
#   # if os.path.exists(parentFolder+'/'+foldername):
#   #   os.chdir(foldername)
#   # else:
#   #   os.mkdir(foldername)
#   #   os.chdir(foldername)
#   # path_fol = Label+'-'+path.split('/')[-1].split('.')[0]
#   # print(os.getcwd())
#   # if os.path.exists(parentFolder+'/'+path_fol):
#   #   os.chdir(path_fol)
#   # else:
#   #   # os.rmdir(path_fol)
#   #   os.mkdir(path_fol)
#   #   os.chdir(path_fol)
#   video = cv2.VideoCapture(path)
#   frame_num = int(video.get(cv2.CAP_PROP_FRAME_COUNT))
#   print(frame_num)
#   print(int(frame_num /32))
#   f_img = 0
#   print(os.getcwd())
#   count = 0
#   while (count<frame_num):
#       try:
#           success, cap = video.read()
#           if count % (int(frame_num / 32)) == 0 and (f_img<32):
#               # print(count)
#               plt.imshow(cap)
#               plt.show()
#               # cv2.imwrite(f" {Label+'-'+path.split('/')[-1].split('.')[0]} frame%d.jpg" % f_img, cap) # save frame as JPEG file.
#               f_img =f_img+1
#               print(f_img)
#               count = count+1
#           else:
#               count = count+1

#       except:
#         count = count+1
#   print(f'done dong doing {path}')


In [ ]:
# # import tqdm
# tqdm.tqdm(listframe(testdf.Path[15], testdf.Label[15]))

In [ ]:
# testpath = '/content/drive/MyDrive/OriginalFolderTest'
# trainpath = '/content/drive/MyDrive/OriginalFramesTrain'

In [ ]:
# test_dirs = os.listdir(testpath)
# train_dirs = os.listdir(trainpath)

In [ ]:
# import random
# # random.shuffle(test_dirs)
# random.shuffle(train_dirs)


In [ ]:
# cur_dirs = []

In [ ]:
# for each in train_dirs:
#   for path_fol in os.listdir(trainpath+'/'+each):
#       os.chdir(trainpath+'/'+each+'/'+path_fol)
#       cur_dir =  os.getcwd()
#       cur_dirs.append(cur_dir)



In [ ]:
# for each in test_dirs:
#   for path_fol in os.listdir(testpath+'/'+each):
#       os.chdir(testpath+'/'+each+'/'+path_fol)
#       cur_dir =  os.getcwd()
#       cur_dirs.append(cur_dir)



In [ ]:
# print(cur_dirs)
# random.shuffle(cur_dirs)

In [ ]:
# images_path = []
# labels = []

# for each in cur_dirs:
#   os.chdir(each)
#   for k in os.listdir(each):
#     images_path.append(each+'/'+k)
#     labels.append(each.split('/')[-2])


In [ ]:
# import pandas as pd
# df = pd.DataFrame(list(zip(images_path, labels)), columns = ['path', 'label'])

In [ ]:
# len(df) /64

In [ ]:
# df = df.sample(frac=1)

In [ ]:
# os.chdir('/content/drive/MyDrive/dataset')

In [ ]:
# df.to_csv('origianltrain_image_path.csv')

In [ ]:
# df.to_csv('origianltest_image_path.csv')

In [ ]:
# !pwd

In [ ]:
# len(df) / 32